In [1]:
# useful libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


Bad key "text.kerning_factor" on line 4 in
/Users/eyobmanhardt/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
import os, sys
import dropbox
#import pandas as pd
from zipfile import ZipFile
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))
from utils.data_extract_utils import extract_zip, extract_features_from_bureau

# say someting ....
# sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(''))))

# Connect to dropbox
dbx = dropbox.Dropbox('cHV7yAR0J6YAAAAAAAAAAVQ1NLCrOwerbaNltPWHslYXKuUTJ5_wfgJsuFcmx83o')

data = {}
for entry in dbx.files_list_folder('').entries:
    response = dbx.files_download('/{}'.format(entry.name))
    
    if 'zip' in entry.name:
        content = extract_zip(response[1].content)
    
        for file in content:
            df = pd.read_csv(file[1])
            data[entry.name.replace('.csv.zip', '')] = df

In [87]:
application = data['application_train'].append(data['application_test']).reset_index()

In [4]:
def get_missing_data_info(df):
    """
    add description .. 
    
    """
    num_of_instance, _ = df.shape
    #print(num_of_instance)
    names = {'index': 'feature', 
             0: 'PERC_missing_data'}
    
    missing_df = (df.isnull() \
        .sum(axis=0)/num_of_instance) \
        .to_frame() \
        .reset_index() \
        .rename(columns=names) \
        .sort_values(by='PERC_missing_data', ascending=False) \
        .reset_index(drop=True)
    
    d = {}
    # no missing 
    zero = missing_df[missing_df['PERC_missing_data'] == 0]
    d['x = 0'] = list(zero.feature)
    le_10 = missing_df[(missing_df['PERC_missing_data'] > 0.0) & (missing_df['PERC_missing_data'] <= 0.1)]
    
    d['0< x <= 25'] = list(missing_df[(missing_df['PERC_missing_data'] > 0.0) & (missing_df['PERC_missing_data'] <= 0.25)].feature)
    
    d['25 < x <= 50'] = list(missing_df[(missing_df['PERC_missing_data'] > 0.25) & (missing_df['PERC_missing_data'] <= 0.5)].feature)
    
    d['x > 50'] = list(missing_df[missing_df['PERC_missing_data'] > 0.5].feature)
    
    return missing_df, d

In [7]:
cols = [i for i in list(application.columns) if i != 'TARGET']
if 'TARGET' in cols:
    print('ahhh')
else:
    print('good')

good


In [94]:
d['x > 50']

['COMMONAREA_MODE',
 'COMMONAREA_AVG',
 'COMMONAREA_MEDI',
 'NONLIVINGAPARTMENTS_MODE',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAPARTMENTS_MEDI',
 'FONDKAPREMONT_MODE',
 'LIVINGAPARTMENTS_MEDI',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAPARTMENTS_MODE',
 'FLOORSMIN_MODE',
 'FLOORSMIN_AVG',
 'FLOORSMIN_MEDI',
 'YEARS_BUILD_MEDI',
 'YEARS_BUILD_MODE',
 'YEARS_BUILD_AVG',
 'OWN_CAR_AGE',
 'LANDAREA_MEDI',
 'LANDAREA_MODE',
 'LANDAREA_AVG',
 'BASEMENTAREA_MODE',
 'BASEMENTAREA_MEDI',
 'BASEMENTAREA_AVG',
 'NONLIVINGAREA_AVG',
 'NONLIVINGAREA_MEDI',
 'NONLIVINGAREA_MODE',
 'EXT_SOURCE_1',
 'ELEVATORS_MEDI',
 'ELEVATORS_MODE',
 'ELEVATORS_AVG',
 'WALLSMATERIAL_MODE',
 'APARTMENTS_MODE',
 'APARTMENTS_MEDI',
 'APARTMENTS_AVG',
 'ENTRANCES_AVG',
 'ENTRANCES_MODE',
 'ENTRANCES_MEDI']

In [8]:
app_missing_info, d = get_missing_data_info(application.loc[:,cols])
for col in d['x > 50']:
    print(col)

COMMONAREA_MODE
COMMONAREA_AVG
COMMONAREA_MEDI
NONLIVINGAPARTMENTS_MODE
NONLIVINGAPARTMENTS_AVG
NONLIVINGAPARTMENTS_MEDI
FONDKAPREMONT_MODE
LIVINGAPARTMENTS_MEDI
LIVINGAPARTMENTS_AVG
LIVINGAPARTMENTS_MODE
FLOORSMIN_MODE
FLOORSMIN_AVG
FLOORSMIN_MEDI
YEARS_BUILD_MEDI
YEARS_BUILD_MODE
YEARS_BUILD_AVG
OWN_CAR_AGE
LANDAREA_MEDI
LANDAREA_MODE
LANDAREA_AVG
BASEMENTAREA_MODE
BASEMENTAREA_MEDI
BASEMENTAREA_AVG
NONLIVINGAREA_AVG
NONLIVINGAREA_MEDI
NONLIVINGAREA_MODE
EXT_SOURCE_1
ELEVATORS_MEDI
ELEVATORS_MODE
ELEVATORS_AVG
WALLSMATERIAL_MODE
APARTMENTS_MODE
APARTMENTS_MEDI
APARTMENTS_AVG
ENTRANCES_AVG
ENTRANCES_MODE
ENTRANCES_MEDI


- remove all feature with missing values > 50
- first aggergate the mean and sum of all EXT_SOURCE

In [17]:
tt1 = application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis = 1)
tt1 = tt1.to_frame()

In [11]:
356076/356255

0.9994975509115661

In [14]:
tt1.head()

,0
0,0.161787
1,0.466757
2,0.642739
3,0.650442
4,0.322738


In [15]:
tt1.isnull().sum()

0    179
dtype: int64

In [18]:
tt2 = application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].sum(axis = 1)
tt2 = tt2.to_frame()
tt2.head()

,0
0,0.485361
1,0.933513
2,1.285479
3,0.650442
4,0.322738


In [19]:
application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].head()

,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3
0,0.083037,0.262949,0.139376
1,0.311267,0.622246,NaN
2,NaN,0.555912,0.729567
3,NaN,0.650442,NaN
4,NaN,0.322738,NaN


## categorical 

In [20]:
cat = list(application.select_dtypes(include = [object]).columns)

In [21]:
cat[0]

'NAME_CONTRACT_TYPE'

In [24]:
application.loc[:, cat[0]].value_counts(dropna=False)

Cash loans         326537
Revolving loans     29718
Name: NAME_CONTRACT_TYPE, dtype: int64

In [28]:
for cc in cat:
    num = application.loc[:, cc].nunique()
    if num  == 2:
        print('Binary:', cc)
    else:
        print('Non-Binary', cc, num)


Binary: NAME_CONTRACT_TYPE
Non-Binary CODE_GENDER 3
Binary: FLAG_OWN_CAR
Binary: FLAG_OWN_REALTY
Non-Binary NAME_TYPE_SUITE 7
Non-Binary NAME_INCOME_TYPE 8
Non-Binary NAME_EDUCATION_TYPE 5
Non-Binary NAME_FAMILY_STATUS 6
Non-Binary NAME_HOUSING_TYPE 6
Non-Binary OCCUPATION_TYPE 18
Non-Binary WEEKDAY_APPR_PROCESS_START 7
Non-Binary ORGANIZATION_TYPE 58
Non-Binary FONDKAPREMONT_MODE 4
Non-Binary HOUSETYPE_MODE 3
Non-Binary WALLSMATERIAL_MODE 7
Binary: EMERGENCYSTATE_MODE


In [ ]:
['FONDKAPREMONT_MODE', 'HOUSETYPE_MODE', 'WALLSMATERIAL_MODE', 'EMERGENCYSTATE_MODE']

In [30]:
binary = ['NAME_CONTRACT_TYPE', 'FLAG_OWN_CAR', 
          'FLAG_OWN_REALTY', 'EMERGENCYSTATE_MODE']
for cc in binary:
    print(application.loc[:, cc].value_counts(dropna=False))
    print('\n')


Cash loans         326537
Revolving loans     29718
Name: NAME_CONTRACT_TYPE, dtype: int64


N    235235
Y    121020
Name: FLAG_OWN_CAR, dtype: int64


Y    246970
N    109285
Name: FLAG_OWN_REALTY, dtype: int64


No     185607
NaN    167964
Yes      2684
Name: EMERGENCYSTATE_MODE, dtype: int64




In [ ]:
## drop EMERGENCYSTATE_MODE

In [32]:
application.shape

(356255, 123)

In [42]:
tt = application.loc[:, 'NAME_TYPE_SUITE'].value_counts(dropna=False).to_frame().reset_index()
tt.columns = ['values', 'count']
tt['prec of val'] = tt['count'].apply(lambda x: x/application.shape[0])
tt.head(8)

,values,count,prec of val
0,Unaccompanied,288253,0.809120
1,Family,46030,0.129205
2,"Spouse, partner",12818,0.035980
3,Children,3675,0.010316
4,NaN,2203,0.006184
5,Other_B,1981,0.005561
6,Other_A,975,0.002737
7,Group of people,320,0.000898


In [33]:
288253/356255

0.8091198720017965

In [34]:
1981/356255

0.005560623710544413

In [43]:
'NAME_TYPE_SUITE'

tt.replace({'values': ['Children', 'Other_B', 
                       'Other_A', 'Group of people']}, 'other')

,values,count,prec of val
0,Unaccompanied,288253,0.809120
1,Family,46030,0.129205
2,"Spouse, partner",12818,0.035980
3,other,3675,0.010316
4,NaN,2203,0.006184
5,other,1981,0.005561
6,other,975,0.002737
7,other,320,0.000898


In [45]:
tt1 = application.loc[:, 'NAME_INCOME_TYPE'].value_counts(dropna=False).to_frame().reset_index()
tt1.columns = ['values', 'count']
tt1['prec of val'] = tt1['count'].apply(lambda x: x/application.shape[0])
tt1.head(8)

,values,count,prec of val
0,Working,183307,0.514539
1,Commercial associate,83019,0.233033
2,Pensioner,64635,0.181429
3,State servant,25235,0.070834
4,Unemployed,23,0.000065
5,Student,20,0.000056
6,Businessman,11,0.000031
7,Maternity leave,5,0.000014


- from unemployed --> mat_leave makes up less than 1% data 

In [ ]:
'NAME_INCOME_TYPE'
['Unemployed', 'Student', 'Businessman', 'Maternity leave']

In [46]:
tt2 = application.loc[:, 'NAME_EDUCATION_TYPE'].value_counts(dropna=False).to_frame().reset_index()
tt2.columns = ['values', 'count']
tt2['prec of val'] = tt2['count'].apply(lambda x: x/application.shape[0])
tt2.head(8)

,values,count,prec of val
0,Secondary / secondary special,252379,0.708422
1,Higher education,87379,0.245271
2,Incomplete higher,12001,0.033687
3,Lower secondary,4291,0.012045
4,Academic degree,205,0.000575


- Combine higher, incomplete higher, and Acadmic degree --> attempted higher education 
- combine Secondary / secondary special and Lower secondary	 --> have sometype of licience 


In [ ]:
def combine(x):
    if x == "incomplete higher" or x == 'Academic degree':
        return 'Higher education'
    else:
        return 'Secondary / secondary special'

In [47]:
tt3 = application.loc[:, 'NAME_FAMILY_STATUS'].value_counts(dropna=False).to_frame().reset_index()
tt3.columns = ['values', 'count']
tt3['prec of val'] = tt3['count'].apply(lambda x: x/application.shape[0])
tt3.head(8)

,values,count,prec of val
0,Married,228715,0.641998
1,Single / not married,52480,0.147310
2,Civil marriage,34036,0.095538
3,Separated,22725,0.063789
4,Widow,18297,0.051359
5,Unknown,2,0.000006


In [48]:
tt4 = application.loc[:, 'NAME_HOUSING_TYPE'].value_counts(dropna=False).to_frame().reset_index()
tt4.columns = ['values', 'count']
tt4['prec of val'] = tt4['count'].apply(lambda x: x/application.shape[0])
tt4.head(8)

,values,count,prec of val
0,House / apartment,316513,0.888445
1,With parents,17074,0.047926
2,Municipal apartment,12800,0.035929
3,Rented apartment,5599,0.015716
4,Office apartment,3024,0.008488
5,Co-op apartment,1245,0.003495


- combine (Municipal apartment, Rented apartment, Office apartment, Co-op apartmen) ---> rented apartment 

In [ ]:
'NAME_HOUSING_TYPE'
['Municipal apartment', 'Office apartment', 'Co-op apartment']

In [51]:
tt5 = application.loc[:, 'OCCUPATION_TYPE'].value_counts(dropna=False).to_frame().reset_index()
tt5.columns = ['values', 'count']
tt5['prec of val'] = tt5['count'].apply(lambda x: x/application.shape[0])
tt5.head(20)

,values,count,prec of val
0,NaN,111996,0.314370
1,Laborers,63841,0.179200
2,Sales staff,37174,0.104347
3,Core staff,31931,0.089630
4,Managers,24945,0.070020
5,Drivers,21376,0.060002
6,High skill tech staff,13234,0.037148
7,Accountants,11441,0.032115
8,Medicine staff,9853,0.027657
9,Security staff,7636,0.021434


In [73]:
other_occp = list(tt5.loc[tt5['prec of val'] < 0.03, 'values'].values)
other_occp

['Medicine staff',
 'Security staff',
 'Cooking staff',
 'Cleaning staff',
 'Private service staff',
 'Low-skill Laborers',
 'Waiters/barmen staff',
 'Secretaries',
 'Realty agents',
 'HR staff',
 'IT staff']

In [52]:
tt6 = application.loc[:, 'ORGANIZATION_TYPE'].value_counts(dropna=False).to_frame().reset_index()
tt6.columns = ['values', 'count']
tt6['prec of val'] = tt6['count'].apply(lambda x: x/application.shape[0])
tt6.head(20)

,values,count,prec of val
0,Business Entity Type 3,78832,0.221280
1,XNA,64648,0.181466
2,Self-employed,44332,0.124439
3,Other,19390,0.054427
4,Medicine,12909,0.036235
5,Business Entity Type 2,12032,0.033774
6,Government,11912,0.033437
7,School,10180,0.028575
8,Trade: type 7,9134,0.025639
9,Kindergarten,7918,0.022226


- orgnization type < 1% of data merge with other 

In [61]:
tt6.loc[tt6['prec of val'] < 0.02, 'prec of val'].sum()

0.21672116882570075

In [62]:
0.21672116882570075+0.05

0.26672116882570074

In [71]:
## make this into a function 
other_org = list(tt6.loc[tt6['prec of val'] < 0.02, 'values'].values)
other_occp = list(tt5.loc[tt5['prec of val'] < 0.03, 'values'].values)

In [86]:
def get_columns_x(col_name, threshhold):
    temp = application.loc[:, col_name].value_counts(dropna=False).to_frame().reset_index()
    temp.columns = ['values', 'count']
    temp['prec of val'] = temp['count'].apply(lambda x: x/application.shape[0])
    groups = list(temp.loc[temp['prec of val'] <threshhold, 'values'].values)
    
    return groups

In [88]:
oc = get_columns_x('OCCUPATION_TYPE', 0.03)

In [89]:
oc =

True

In [80]:
# helper functions
def get_missing_data_info(df):
    """
    add description .. 
    
    """
    num_of_instance, _ = df.shape
    #print(num_of_instance)
    names = {'index': 'feature', 
             0: 'PERC_missing_data'}
    
    missing_df = (df.isnull() \
        .sum(axis=0)/num_of_instance) \
        .to_frame() \
        .reset_index() \
        .rename(columns=names) \
        .sort_values(by='PERC_missing_data', ascending=False) \
        .reset_index(drop=True)
    
    d = {}
    # no missing 
    zero = missing_df[missing_df['PERC_missing_data'] == 0]
    d['x = 0'] = list(zero.feature)
    le_10 = missing_df[(missing_df['PERC_missing_data'] > 0.0) & (missing_df['PERC_missing_data'] <= 0.1)]
    
    d['0< x <= 25'] = list(missing_df[(missing_df['PERC_missing_data'] > 0.0) & (missing_df['PERC_missing_data'] <= 0.25)].feature)
    
    d['25 < x <= 50'] = list(missing_df[(missing_df['PERC_missing_data'] > 0.25) & (missing_df['PERC_missing_data'] <= 0.5)].feature)
    
    d['x > 50'] = list(missing_df[missing_df['PERC_missing_data'] > 0.5].feature)
    
    return missing_df, d

def combine(x):
    if x == "Incomplete higher" or x == 'Academic degree':
        return 'Higher education'
    elif x == 'Lower secondary':
        return 'Secondary / secondary special'
    else:
        return x
    
def get_columns_x(col_name, threshhold):
    temp = application.loc[:, col_name].value_counts(dropna=False).to_frame().reset_index()
    temp.columns = ['values', 'count']
    temp['prec of val'] = temp['count'].apply(lambda x: x/application.shape[0])
    groups = list(temp.loc[temp['prec of val'] <threshhold, 'values'].values)
    
    return groups

def corr_with_target(application):
    target = application.loc[application['TARGET'].notnull(), 'TARGET']
    train_df = application.loc[application['TARGET'].notnull(), :].drop(columns=['TARGET']).select_dtypes(include = [int, float])

    corr_col = [i for i in train_df.columns if i not in ['index', 'SK_ID_CURR']]
    corr_arr = pd.Series([target.corr(train_df[i]) for i in corr_col])
    
    name = pd.Series(corr_col)
    df_dict = {'feature': name, 'corr with target': corr_arr}
    corr_df = pd.DataFrame(df_dict).sort_values(by = 'corr with target',ascending=False)
    
    return corr_df

In [90]:

# fix categorical data 

# 'NAME_TYPE_SUITE'
application.replace({'NAME_TYPE_SUITE': ['Children', 
                                         'Other_B', 
                                         'Other_A', 
                                         'Group of people']}, 
                    'Other', inplace = True)


# 'NAME_INCOME_TYPE'
application.replace({'NAME_INCOME_TYPE': ['Unemployed', 
                                          'Student', 
                                          'Businessman', 
                                          'Maternity leave']}, 
                    'Other', inplace=True)

# NAME_EDUCATION_TYPE
application['NAME_EDUCATION_TYPE'] = application['NAME_EDUCATION_TYPE'].apply(lambda x: combine(x))

#'NAME_HOUSING_TYPE'
application.replace({'NAME_HOUSING_TYPE': ['Municipal apartment', 
                                           'Office apartment', 
                                           'Co-op apartment']}, 
                    'Rented apartment', inplace = True)

# OCCUPATION_TYPE
application.replace({'OCCUPATION_TYPE': other_occp}, 
                    'Other', inplace = True)

# ORGANIZATION_TYPE 
application.replace({'ORGANIZATION_TYPE': other_org}, 
                    'Other', inplace = True)

# drop cat features - low variance and high % of missing val
application.drop(columns = ['FONDKAPREMONT_MODE', 
                            'HOUSETYPE_MODE', 
                            'WALLSMATERIAL_MODE', 
                            'EMERGENCYSTATE_MODE'], 
                 axis = 1, inplace=True)

## label encoder for binary values
bin_features = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

replace_dict = {'CODE_GENDER': {'M': 0, 'F': 1},
                'FLAG_OWN_CAR': {'Y': 0, 'N': 1}, 
                'FLAG_OWN_REALTY':{'Y': 0, 'N': 1},
               'NAME_EDUCATION_TYPE': {'Higher education': 1,
                                      'Secondary / secondary special': 0}}

application.replace(replace_dict, inplace=True)


# remove few instances
application = application.loc[application['CODE_GENDER'] != 'XNA', :]
application = application.loc[application['NAME_FAMILY_STATUS'] != 'Unknown', :]


# drop_features > 50% of missing vals (both: cat, num)
cols = [i for i in list(application.columns) if i != 'TARGET']
_, missing_dict = get_missing_data_info(application.loc[:,cols])
drop_cols = missing_dict['x > 50']

# keep EXT_SOURCE_1 --> highly corr with target 
drop_cols.remove('EXT_SOURCE_1')

application.drop(columns = drop_cols, axis = 1, inplace = True)

# Numerical features 

## DAYS_EMPLOYED
application['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
application['DAYS_EMPLOYED'] =application['DAYS_EMPLOYED'].apply(lambda x: abs(x))

application['BIRTH_IN_YEARS'] = application['DAYS_BIRTH'].apply(lambda x: abs(x)/365)



# # remove few instances
# application = application.loc[application['CODE_GENDER'] != 'XNA', :]
# application = application.loc[application['NAME_FAMILY_STATUS'] != 'Unknown', :]


# add new features 
application['ALL_EXT_SOURCE_MEAN']=application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis = 1)
application['PAYMENT_RATE'] = application['AMT_ANNUITY']/application['AMT_CREDIT']
application['INCOME_PER_PERSON'] = application['AMT_INCOME_TOTAL']/application['CNT_FAM_MEMBERS']
application['INCOME_CREDIT_IN_PERCENTAGE '] = application['AMT_INCOME_TOTAL']/application['AMT_CREDIT']
application['ANNUITY_INCOME_IN_PERCENTAGE'] = application['AMT_ANNUITY']/application['AMT_INCOME_TOTAL']

doc_cols = [i for i in list(application.columns) if 'FLAG_DOCUMENT' in i]
application['ALL_FLAG_DOCUMENT_SUM']=application[doc_cols].mean(axis = 1)

## drop indiv doc columns 
application.drop(columns = doc_cols, axis = 1, inplace = True)

if 'index' in list(application.columns):
    application.drop(columns = ['index'], axis = 1, inplace = True)
    print('index column removed')


## label encoder for binary values
# bin_features = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

# replace_dict = {'CODE_GENDER': {'M': 0, 'F': 1},
#                 'FLAG_OWN_CAR': {'Y': 0, 'N': 1}, 
#                 'FLAG_OWN_REALTY':{'Y': 0, 'N': 1} }
# application.replace(replace_dict, inplace=True)

In [77]:
application.loc[:, 'NAME_TYPE_SUITE'].value_counts(dropna=False).to_frame().reset_index()

,index,NAME_TYPE_SUITE
0,Unaccompanied,288250
1,Family,46029
2,"Spouse, partner",12818
3,Other,6951
4,NaN,2201


In [78]:
application.loc[:, 'NAME_INCOME_TYPE'].value_counts(dropna=False).to_frame().reset_index()

,index,NAME_INCOME_TYPE
0,Working,183304
1,Commercial associate,83016
2,Pensioner,64635
3,State servant,25235
4,Other,59


In [91]:
application.loc[:, 'NAME_EDUCATION_TYPE'].value_counts(dropna=False).to_frame().reset_index()

,index,NAME_EDUCATION_TYPE
0,0,256667
1,1,99582


In [81]:
application.loc[:, 'NAME_HOUSING_TYPE'].value_counts(dropna=False).to_frame().reset_index()

,index,NAME_HOUSING_TYPE
0,House / apartment,316509
1,Rented apartment,22666
2,With parents,17074


In [82]:
application.loc[:, 'OCCUPATION_TYPE'].value_counts(dropna=False).to_frame().reset_index()

,index,OCCUPATION_TYPE
0,NaN,111994
1,Laborers,63841
2,Other,40316
3,Sales staff,37174
4,Core staff,31930
5,Managers,24944
6,Drivers,21376
7,High skill tech staff,13234
8,Accountants,11440


In [84]:
application.loc[:, 'ORGANIZATION_TYPE'].value_counts(dropna=False).to_frame().reset_index()

,index,ORGANIZATION_TYPE
0,Other,96595
1,Business Entity Type 3,78832
2,XNA,64648
3,Self-employed,44332
4,Medicine,12908
5,Business Entity Type 2,12031
6,Government,11912
7,School,10180
8,Trade: type 7,9134
9,Kindergarten,7917


In [93]:
Xt = application.loc[application['TARGET'].isnull(), :].drop(columns=['TARGET'], axis=1)
Xt.shape

(48744, 124)

In [98]:
doc_cols = [i for i in list(application.columns) if 'FLAG_DOCUMENT' in i]
doc_cols

['FLAG_DOCUMENT_2',
 'FLAG_DOCUMENT_3',
 'FLAG_DOCUMENT_4',
 'FLAG_DOCUMENT_5',
 'FLAG_DOCUMENT_6',
 'FLAG_DOCUMENT_7',
 'FLAG_DOCUMENT_8',
 'FLAG_DOCUMENT_9',
 'FLAG_DOCUMENT_10',
 'FLAG_DOCUMENT_11',
 'FLAG_DOCUMENT_12',
 'FLAG_DOCUMENT_13',
 'FLAG_DOCUMENT_14',
 'FLAG_DOCUMENT_15',
 'FLAG_DOCUMENT_16',
 'FLAG_DOCUMENT_17',
 'FLAG_DOCUMENT_18',
 'FLAG_DOCUMENT_19',
 'FLAG_DOCUMENT_20',
 'FLAG_DOCUMENT_21']

In [96]:
"FLAG_DOCUMENT" in "FLAG_DOCUMENT_2"

True

In [97]:
list(application.columns)

['index',
 'SK_ID_CURR',
 'TARGET',
 'NAME_CONTRACT_TYPE',
 'CODE_GENDER',
 'FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_TYPE_SUITE',
 'NAME_INCOME_TYPE',
 'NAME_EDUCATION_TYPE',
 'NAME_FAMILY_STATUS',
 'NAME_HOUSING_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'OCCUPATION_TYPE',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'WEEKDAY_APPR_PROCESS_START',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'ORGANIZATION_TYPE',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPL

In [99]:
application.loc[:, 'FLAG_EMAIL'].value_counts()

0    330879
1     25370
Name: FLAG_EMAIL, dtype: int64

In [100]:
application.loc[:, 'FLAG_MOBIL'].value_counts()

1    356247
0         2
Name: FLAG_MOBIL, dtype: int64

In [101]:
application.loc[:, 'FLAG_EMP_PHONE'].value_counts()

1    291588
0     64661
Name: FLAG_EMP_PHONE, dtype: int64

In [102]:
application.loc[:, 'FLAG_WORK_PHONE'].value_counts()

0    284968
1     71281
Name: FLAG_WORK_PHONE, dtype: int64

In [103]:
application.loc[:, 'FLAG_CONT_MOBILE'].value_counts()

1    355597
0       652
Name: FLAG_CONT_MOBILE, dtype: int64

In [104]:
application.loc[:, 'FLAG_PHONE'].value_counts()

0    256997
1     99252
Name: FLAG_PHONE, dtype: int64

In [110]:
target = application.loc[application['TARGET'].notnull(), 'TARGET']
train_df = application.loc[application['TARGET'].notnull(), :].drop(columns=['TARGET']).select_dtypes(include = [int, float])

In [113]:
target.corr(train_df['FLAG_OWN_CAR'])

0.021851250991688637

In [115]:
corr_col = [i for i in train_df.columns if i not in ['index', 'SK_ID_CURR']]
corr_col

['FLAG_OWN_CAR',
 'FLAG_OWN_REALTY',
 'CNT_CHILDREN',
 'AMT_INCOME_TOTAL',
 'AMT_CREDIT',
 'AMT_ANNUITY',
 'AMT_GOODS_PRICE',
 'NAME_EDUCATION_TYPE',
 'REGION_POPULATION_RELATIVE',
 'DAYS_BIRTH',
 'DAYS_EMPLOYED',
 'DAYS_REGISTRATION',
 'DAYS_ID_PUBLISH',
 'OWN_CAR_AGE',
 'FLAG_MOBIL',
 'FLAG_EMP_PHONE',
 'FLAG_WORK_PHONE',
 'FLAG_CONT_MOBILE',
 'FLAG_PHONE',
 'FLAG_EMAIL',
 'CNT_FAM_MEMBERS',
 'REGION_RATING_CLIENT',
 'REGION_RATING_CLIENT_W_CITY',
 'HOUR_APPR_PROCESS_START',
 'REG_REGION_NOT_LIVE_REGION',
 'REG_REGION_NOT_WORK_REGION',
 'LIVE_REGION_NOT_WORK_REGION',
 'REG_CITY_NOT_LIVE_CITY',
 'REG_CITY_NOT_WORK_CITY',
 'LIVE_CITY_NOT_WORK_CITY',
 'EXT_SOURCE_1',
 'EXT_SOURCE_2',
 'EXT_SOURCE_3',
 'APARTMENTS_AVG',
 'BASEMENTAREA_AVG',
 'YEARS_BEGINEXPLUATATION_AVG',
 'YEARS_BUILD_AVG',
 'COMMONAREA_AVG',
 'ELEVATORS_AVG',
 'ENTRANCES_AVG',
 'FLOORSMAX_AVG',
 'FLOORSMIN_AVG',
 'LANDAREA_AVG',
 'LIVINGAPARTMENTS_AVG',
 'LIVINGAREA_AVG',
 'NONLIVINGAPARTMENTS_AVG',
 'NONLIVINGAREA_AVG

In [128]:
corr_arr = pd.Series([target.corr(train_df[i]) for i in corr_col])

In [130]:
corr_ar 

0      0.021851
1      0.006145
2      0.019188
3     -0.003980
4     -0.030371
         ...   
108   -0.222053
109    0.012703
110   -0.006572
111   -0.001816
112    0.014261
Length: 113, dtype: float64

In [136]:
corr_arr = pd.Series([target.corr(train_df[i]) for i in corr_col])
name = pd.Series(corr_col)
df_dict = {'feature': name, 'corr with target': corr_arr}
corr_df = pd.DataFrame(df_dict).sort_values(by = 'corr with target',ascending=False)
corr_df.head()

,feature,corr with target
9,DAYS_BIRTH,0.078244
22,REGION_RATING_CLIENT_W_CITY,0.060895
21,REGION_RATING_CLIENT,0.058901
80,DAYS_LAST_PHONE_CHANGE,0.055220
12,DAYS_ID_PUBLISH,0.051457


In [137]:
corr_df.tail()

,feature,corr with target
107,BIRTH_IN_YEARS,-0.078244
30,EXT_SOURCE_1,-0.155316
31,EXT_SOURCE_2,-0.160470
32,EXT_SOURCE_3,-0.178926
108,ALL_EXT_SOURCE_MEAN,-0.222053


In [131]:
name

0                      FLAG_OWN_CAR
1                   FLAG_OWN_REALTY
2                      CNT_CHILDREN
3                  AMT_INCOME_TOTAL
4                        AMT_CREDIT
                   ...             
108             ALL_EXT_SOURCE_MEAN
109                    PAYMENT_RATE
110               INCOME_PER_PERSON
111    INCOME_CREDIT_IN_PERCENTAGE 
112    ANNUITY_INCOME_IN_PERCENTAGE
Length: 113, dtype: object

In [126]:
name = pd.Series(corr_col)

In [132]:
corr_ar .to_frame().join(name)

ValueError: Other Series must have a name

In [124]:
corr_ar

0      0.021851
1      0.006145
2      0.019188
3     -0.003980
4     -0.030371
         ...   
108   -0.222053
109    0.012703
110   -0.006572
111   -0.001816
112    0.014261
Length: 113, dtype: float64

In [119]:
corr_ar

[-0.2220528669193609,
 -0.17892558500020928,
 -0.1604701488076733,
 -0.15531608150734533,
 -0.07824384822893445,
 -0.07495854303154034,
 -0.05338930271475458,
 -0.044006878686108294,
 -0.043771394234479774,
 -0.04322957891465342,
 -0.03964669449611322,
 -0.03722458661814109,
 -0.034202272890432595,
 -0.03386630597702336,
 -0.03361598977570253,
 -0.033396745746269614,
 -0.03300117368038642,
 -0.03274329971622148,
 -0.032700098602900136,
 -0.03259925892645475,
 -0.032134426289234075,
 -0.03068837982801088,
 -0.03037113793363464,
 -0.029501849040104135,
 -0.029187996511464944,
 -0.02860374216456787,
 -0.027287851803987503,
 -0.025034706201995518,
 -0.024624809676221146,
 -0.024162310186414317,
 -0.023799698959328545,
 -0.023397154438522774,
 -0.02274792583194022,
 -0.02232204091081997,
 -0.022145387174521795,
 -0.02208339584413382,
 -0.022064168994454566,
 -0.019954047075780287,
 -0.019176869271546566,
 -0.01902938913175245,
 -0.018575852748632018,
 -0.018552659744450924,
 -0.017391718707

In [138]:
def corr_with_target(application):
    target = application.loc[application['TARGET'].notnull(), 'TARGET']
    train_df = application.loc[application['TARGET'].notnull(), :].drop(columns=['TARGET']).select_dtypes(include = [int, float])

    corr_col = [i for i in train_df.columns if i not in ['index', 'SK_ID_CURR']]
    corr_arr = pd.Series([target.corr(train_df[i]) for i in corr_col])
    
    name = pd.Series(corr_col)
    df_dict = {'feature': name, 'corr with target': corr_arr}
    corr_df = pd.DataFrame(df_dict).sort_values(by = 'corr with target',ascending=False)
    
    return corr_df

In [139]:
xx = corr_with_target(application)

In [140]:
xx.head()

,feature,corr with target
9,DAYS_BIRTH,0.078244
22,REGION_RATING_CLIENT_W_CITY,0.060895
21,REGION_RATING_CLIENT,0.058901
80,DAYS_LAST_PHONE_CHANGE,0.055220
12,DAYS_ID_PUBLISH,0.051457


In [141]:
yy = [1,2,4].remove(4)
yy

In [142]:
yy

## Take away from notebook

In [ ]:
# helper functions
def get_missing_data_info(df):
    """
    add description .. 
    
    """
    num_of_instance, _ = df.shape
    #print(num_of_instance)
    names = {'index': 'feature', 
             0: 'PERC_missing_data'}
    
    missing_df = (df.isnull() \
        .sum(axis=0)/num_of_instance) \
        .to_frame() \
        .reset_index() \
        .rename(columns=names) \
        .sort_values(by='PERC_missing_data', ascending=False) \
        .reset_index(drop=True)
    
    d = {}
    # no missing 
    zero = missing_df[missing_df['PERC_missing_data'] == 0]
    d['x = 0'] = list(zero.feature)
    le_10 = missing_df[(missing_df['PERC_missing_data'] > 0.0) & (missing_df['PERC_missing_data'] <= 0.1)]
    
    d['0< x <= 25'] = list(missing_df[(missing_df['PERC_missing_data'] > 0.0) & (missing_df['PERC_missing_data'] <= 0.25)].feature)
    
    d['25 < x <= 50'] = list(missing_df[(missing_df['PERC_missing_data'] > 0.25) & (missing_df['PERC_missing_data'] <= 0.5)].feature)
    
    d['x > 50'] = list(missing_df[missing_df['PERC_missing_data'] > 0.5].feature)
    
    return missing_df, d

def combine(x):
    if x == "Incomplete higher" or x == 'Academic degree':
        return 'Higher education'
    elif x == 'Lower secondary':
        return 'Secondary / secondary special'
    else:
        return x
    
def get_columns_x(col_name, threshhold):
    temp = application.loc[:, col_name].value_counts(dropna=False).to_frame().reset_index()
    temp.columns = ['values', 'count']
    temp['prec of val'] = temp['count'].apply(lambda x: x/application.shape[0])
    groups = list(temp.loc[temp['prec of val'] <threshhold, 'values'].values)
    
    return groups

def corr_with_target(application):
    target = application.loc[application['TARGET'].notnull(), 'TARGET']
    train_df = application.loc[application['TARGET'].notnull(), :].drop(columns=['TARGET']).select_dtypes(include = [int, float])

    corr_col = [i for i in train_df.columns if i not in ['index', 'SK_ID_CURR']]
    corr_arr = pd.Series([target.corr(train_df[i]) for i in corr_col])
    
    name = pd.Series(corr_col)
    df_dict = {'feature': name, 'corr with target': corr_arr}
    corr_df = pd.DataFrame(df_dict).sort_values(by = 'corr with target',ascending=False)
    
    return corr_df

In [ ]:
def clean_application(data):
    
    # get table 
    application = data['application_train'].append(data['application_test']).reset_index()
    
    # fix categorical data 

    # 'NAME_TYPE_SUITE'
    application.replace({'NAME_TYPE_SUITE': ['Children', 
                                             'Other_B', 
                                             'Other_A', 
                                             'Group of people']}, 
                        'Other', inplace = True)


    # 'NAME_INCOME_TYPE'
    application.replace({'NAME_INCOME_TYPE': ['Unemployed', 
                                              'Student', 
                                              'Businessman', 
                                              'Maternity leave']}, 
                        'Other', inplace=True)

    # NAME_EDUCATION_TYPE
    application['NAME_EDUCATION_TYPE'] = application['NAME_EDUCATION_TYPE'].apply(lambda x: combine(x))

    #'NAME_HOUSING_TYPE'
    application.replace({'NAME_HOUSING_TYPE': ['Municipal apartment', 
                                               'Office apartment', 
                                               'Co-op apartment']}, 
                        'Rented apartment', inplace = True)

    # OCCUPATION_TYPE
    application.replace({'OCCUPATION_TYPE': other_occp}, 
                        'Other', inplace = True)

    # ORGANIZATION_TYPE 
    application.replace({'ORGANIZATION_TYPE': other_org}, 
                        'Other', inplace = True)

    # drop cat features - low variance and high % of missing val
    application.drop(columns = ['FONDKAPREMONT_MODE', 
                                'HOUSETYPE_MODE', 
                                'WALLSMATERIAL_MODE', 
                                'EMERGENCYSTATE_MODE'], 
                     axis = 1, inplace=True)

    ## label encoder for binary values
    bin_features = ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']

    replace_dict = {'CODE_GENDER': {'M': 0, 'F': 1},
                    'FLAG_OWN_CAR': {'Y': 0, 'N': 1}, 
                    'FLAG_OWN_REALTY':{'Y': 0, 'N': 1},
                   'NAME_EDUCATION_TYPE': {'Higher education': 1,
                                          'Secondary / secondary special': 0}}

    application.replace(replace_dict, inplace=True)


    # remove few instances with low appearance
    application = application.loc[application['CODE_GENDER'] != 'XNA', :]
    application = application.loc[application['NAME_FAMILY_STATUS'] != 'Unknown', :]


    # drop_features > 50% of missing vals (both: cat, num)
    cols = [i for i in list(application.columns) if i != 'TARGET']
    _, missing_dict = get_missing_data_info(application.loc[:,cols])
    drop_cols = missing_dict['x > 50']

    # keep EXT_SOURCE_1 --> highly corr with target 
    drop_cols.remove('EXT_SOURCE_1')
    application.drop(columns = drop_cols, axis = 1, inplace = True)

    # Numerical features 

    ## DAYS_EMPLOYED
    application['DAYS_EMPLOYED'].replace(365243, np.nan, inplace=True)
    application['DAYS_EMPLOYED'] =application['DAYS_EMPLOYED'].apply(lambda x: abs(x))

    application['BIRTH_IN_YEARS'] = application['DAYS_BIRTH'].apply(lambda x: abs(x)/365)


    # add new features 
    application['ALL_EXT_SOURCE_MEAN']=application[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].mean(axis = 1)
    application['PAYMENT_RATE'] = application['AMT_ANNUITY']/application['AMT_CREDIT']
    application['INCOME_PER_PERSON'] = application['AMT_INCOME_TOTAL']/application['CNT_FAM_MEMBERS']
    application['INCOME_CREDIT_IN_PERCENTAGE '] = application['AMT_INCOME_TOTAL']/application['AMT_CREDIT']
    application['ANNUITY_INCOME_IN_PERCENTAGE'] = application['AMT_ANNUITY']/application['AMT_INCOME_TOTAL']

    doc_cols = [i for i in list(application.columns) if 'FLAG_DOCUMENT' in i]
    application['ALL_FLAG_DOCUMENT_SUM']=application[doc_cols].mean(axis = 1)

    ## drop indiv doc columns 
    application.drop(columns = doc_cols, axis = 1, inplace = True)

    if 'index' in list(application.columns):
        application.drop(columns = ['index'], axis = 1, inplace = True)
        print('index column removed')

    return application

